In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Model, load_model
# Input and Concatenate layers
from tensorflow.keras.layers import Input, Concatenate
import pickle


# Custom AttentionLayer definition
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        assert isinstance(inputs, list)
        encoder_out_seq, decoder_out_seq = inputs

        # Attention mechanism
        score = tf.matmul(decoder_out_seq, encoder_out_seq, transpose_b=True)
        attention_weights = tf.nn.softmax(score, axis=-1)
        context_vector = tf.matmul(attention_weights, encoder_out_seq)

        return context_vector, attention_weights

    def compute_output_shape(self, input_shape):
        assert isinstance(input_shape, list)
        shape1 = input_shape[1][:2] + (input_shape[0][1],)
        shape2 = input_shape[1][:2] + (input_shape[0][0],)
        return [shape1, shape2]

# Load tokenizer
with open('y_tokenizer.pickle', 'rb') as handle:
    y_tokenizer = pickle.load(handle)

with open('X_tokenizer.pickle', 'rb') as handle:
    X_tokenizer = pickle.load(handle)

In [14]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=X_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [15]:
#Inference/Validation Phase
#Encoding the input sequence to get the feature vector
max_text_len = 575
embedding_dim = 256
latent_dim = 512
max_summary_len = 75
X_voc = 76409 
encoder_inputs = tf.keras.layers.Input(shape=(max_text_len,))
enc_emb = tf.keras.layers.Embedding(X_voc, embedding_dim, trainable=True)(encoder_inputs)
encoder_lstm = tf.keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

#Decoder setup
#These tensors will hold the states of the previous time step
y_voc = 38333
decoder_inputs = tf.keras.layers.Input(shape=(None,))
dec_emb_layer = tf.keras.layers.Embedding(y_voc, embedding_dim, trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

#Getting the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)

#Setting the initial states to the states from the previous time step for better prediction
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#Attention inference
attn_layer = AttentionLayer()
attn_out, attn_weights = attn_layer([encoder_outputs, decoder_outputs])
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

#Adding Dense softmax layer to generate proability distribution over the target vocabulary
decoder_dense = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(y_voc, activation='softmax'))
decoder_outputs2 = decoder_dense(decoder_inf_concat)

#Final Decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [16]:
#Function defining the implementation of inference process
def decode_sequence(input_seq):
    #Encoding the input as state vectors
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    #Generating empty target sequence of length 1
    target_seq = np.zeros((1,1))

    #Populating the first word of target sequence with the start word
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:

        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        #Sampling a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]

        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        #Exit condition: either hit max length or find stop word
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        #Updating the target sequence (of length 1)
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        #Updating internal states
        e_h, e_c = h, c

    return decoded_sentence

In [17]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [19]:
content = "breezy sweep pen president vladimir putin wrote new chapter crimea turbulent history committing region future returned russian domain sixty years prior ukraine breakaway peninsula signed away swiftly soviet leader nikita khrushchev dealing blatant land grab eastern flank anywhere near quick easy europe member union unlike crimea rushed referendum everyone say initially slapping visa restrictions asset freezes limited number little known politicians military men europe facing urgent calls widen scope measures target russian business community particular logic run russia essentially two sides coin alexei navalny one time moscow mayoral contender house arrest opposing current regime called europe leaders ban everyone vladimir putin personal banker chelsea football club owner roman abramovich keeping money loved ones abroad asset freezes visa restrictions especially palatable options eu rolled discretionary basis without requiring cumbersome legal procedures recourse fact russia cancels visas people like time look hermitage capital founder bill browder lost right entry moscow based money dare go back russia also banned adoption orphans americans retaliation us implementation anti corruption law named sergei magnitsky browder lawyer died year moscow detention center apparently beaten death yet playing money talks card europe must ready consequences action money also walks eu leaders must ready accept sanctions two way street hurt sides targeting russia peripatetic business community would one way sapping tenuous support president putin strategy might also turn silver lining awarding eu countries chance finally deal unpleasant sides patronage including money laundering corruption inflated prize assets like london property picasso paintings years europe hold fire though trade two decades post soviet rapprochement almost billion worth commerce lot put stake true trade war would hurt russia far harder would eu least former gdp comes exports bloc europe hefty reliance russian gas would hard time keeping factories going citizens warm without power east putin flexes political muscle open trade channels keep dialogue going giving sides chance change subject talk less tensely one afford cut lifeline especially europe economy rebound russia one wane"
content_seq = X_tokenizer.texts_to_sequences([content])
content_seq = pad_sequences(content_seq,  maxlen=max_text_len, padding='post')
summary = decode_sequence(content_seq)
print(summary)

1/1 [==============================] - 0s 65ms/step
 bj zion zion goalscoring printing sterling chicken bo pigments dashcam dashcam sentimental perrie ines pta programmed cilic cilic cilic sclerosis enjoy outbid braced kingdom braced kingdom slugger divisions grass divisions zhuang dzhokhar chicken november chicken pastors specialist payoffs respectful goalscoring drafted lupo malformation malformation asprilla asprilla excise dozen segway respectful respectful pecking nafeek drafted lupo injections pigments ruthless addressed etan nastase socialise conquer conquer gamez surfer nafeek misdiagnosed misdiagnosed ruthless reconstruct perrie payoffs lupo
